<a href="https://colab.research.google.com/github/haengeunc/looker_user_maintenance/blob/main/Looker_user_maintenance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

⚡ Welcome to user maintenance tool! ⚡

Do you want to automate a process of identifying and archiving user contents before deleting them from the Looker instance? This tool can help!

Development environment requirements:
*  looker-sdk-22.18.0
*  looker-deployer
*  ruby




# Set up the Looker Python SDK

In [ ]:
!pip install looker-sdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# For the SDK to work, you must either have set environment variables or created a looker.ini in the working directory.
#set variables on the fly - note that API credentials should NOT be left if being shared. 

# 1. Setting environment variables:
import os #We import os here in order to manage environment variables for the tutorial. You don't need to do this on a local system or anywhere you can more conveniently set environment variables.

os.environ["LOOKERSDK_BASE_URL"] = "https://teach.corp.looker.com:19999" #If your looker URL has .cloud in it (hosted on GCP), do not include :19999 (ie: https://your.cloud.looker.com).
os.environ["LOOKERSDK_VERIFY_SSL"] = "true" #Defaults to true if not set. SSL verification should generally be on unless you have a real good reason not to use it. Valid options: true, y, t, yes, 1.

#Get the following values from your Users page in the Admin panel of your Looker instance > Users > Your user > Edit API keys. If you know your user id, you can visit https://your.looker.com/admin/users/<your_user_id>/edit.
os.environ["LOOKERSDK_CLIENT_ID"] =  "x5FZYJnhHd4Ns4mxKpJT" #No defaults.
os.environ["LOOKERSDK_CLIENT_SECRET"] = "TqXRkMMskS6sfWPQHjHc9b4z" #No defaults. This should be protected at all costs. Please do not leave it sitting here, even if you don't share this document.

print("All environment variables set for", os.environ["LOOKERSDK_BASE_URL"])

# 2. Alternaitve option is to create looker.ini file in the working directory:
   #[Looker]
   #base_url=https://teach.corp.looker.com:19999
   #client_id=xxx
   #client_secret=yyy
   #verify_ssl=true



All environment variables set for https://teach.corp.looker.com:19999


In [ ]:
import json #This is a handy library for doing JSON work.

In [ ]:
import looker_sdk

# For this to work you must either have set environment variables or created a looker.ini as described below in "Configuring the SDK"
# Initialize Looker SDK 4.0
sdk = looker_sdk.init40()  # or init31() for the older v3.1 API

# Make your first Looker API calls
my_user = sdk.me()

print(my_user.first_name)



Haengeun


# Looker deployer open source tool

In [ ]:
#Refer to https://github.com/looker-open-source/looker_deployer
!pip install looker-deployer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Install ruby
!sudo apt-get install ruby ruby-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ruby is already the newest version (1:2.5.1).
ruby-dev is already the newest version (1:2.5.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 26 not upgraded.


In [ ]:
!gem install public_suffix -v 4.0.7

Parsing documentation for public_suffix-4.0.7
Done installing documentation for public_suffix after 0 seconds
1 gem installed


In [ ]:
!gem install gazer

Parsing documentation for gazer-0.2.56
Done installing documentation for gazer after 0 seconds
1 gem installed


In [ ]:
users_to_delete = ["19371", "19811"]

#Loop through a list of users that need deleting and generate a list of folder numbers
def get_folder_number():
  
  folder_id = []
  
  for uid in users_to_delete:
    
    response = sdk.user(
            user_id= uid, #pass uid here ######***********
            fields="email, personal_folder_id")   
    
    folder_id.append(response['personal_folder_id'])
  
  return folder_id
  
  #print(response['personal_folder_id'])
  #print(type(response['personal_folder_id']))

#run function to print folder ID
folder_list = get_folder_number()
print(folder_list)



['16640', '16986']


In [ ]:
list = ''
for item in folder_list:
  list = list + ' ' + item

print(list)


 16640 16986


In [ ]:
!ldeploy content export -h

usage: ldeploy content export [-h] --env ENV [--ini INI] [--debug] --folders
                              FOLDERS [FOLDERS ...] --local-target
                              LOCAL_TARGET

optional arguments:
  -h, --help            show this help message and exit
  --env ENV             What environment to deploy to
  --ini INI             ini file to parse for credentials
  --debug               set logger to debug for more verbosity
  --folders FOLDERS [FOLDERS ...]
                        What folders to export content from
  --local-target LOCAL_TARGET
                        Local directory to store content


In [ ]:
#run ldeploy to export a content from a specified folder to local disk
!ldeploy content export --env Looker --folders $list --local-target .

{"levelname": "INFO", "module": "deploy_content_export", "funcName": "main", "message": "Exporting content", "env": "ENV", "folders": ["16640", "16986"], "local_target": ".", "timestamp": "2022-10-24T16:36:37.383794Z"}
Traceback (most recent call last):
  File "/usr/local/bin/ldeploy", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/looker_deployer/cli.py", line 46, in main
    args.func(args)
  File "/usr/local/lib/python3.7/dist-packages/looker_deployer/commands/deploy_content_export.py", line 105, in main
    sdk = get_client(args.ini, args.env)
  File "/usr/local/lib/python3.7/dist-packages/looker_deployer/utils/get_client.py", line 42, in get_client
    sdk = configure_sdk(config_file=ini, section=env)
  File "/usr/local/lib/python3.7/dist-packages/looker_deployer/utils/get_client.py", line 27, in configure_sdk
    _settings(config_file, section) if config_settings is None else config_settings
  File "/usr/local/lib/python3.7/dist-packages/l

# Create a Google Cloud Storage (gcs) bucket to save folder contents

In [ ]:
#authenticate with google cloud sdk
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=FzCx2wj49181GOo9DsqauRzAo2x4cM&prompt=consent&access_type=offline&code_challenge=dKjA_ns18H1zP6wZt9TyxnOdqC-9xUzE9lip-Rtw2YI&code_challenge_method=S256

Enter authorization code: 4/0ARtbsJoFnGa21uHSQvl8-FAGxYQ8MZ6SA7Y1IO1CfZqQYbXYV8b2lIwoTyh5iyZPQr9ISA

You are now logged in as [haengeun@google.com].
Your current project is [haengeun].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [ ]:
#set project ID in xxx
#!gcloud config set project xxx
!gcloud config set project haengeun

Updated property [core/project].


In [ ]:
#list all the cloud storage buckets available
!gsutil ls 

gs://dataproc-staging-us-central1-1048293672564-jdjrxdix/
gs://dataproc-temp-us-central1-1048293672564-lgqq5ibd/
gs://e4-terraform-bucket/
gs://haengeun-bucket2/
gs://teach-instance-maintenance/


In [ ]:
#cloud storage - Copy files and objects into 
#gsutil cp -r dir gs://my-bucket/data

!gsutil -m cp -r Users gs://teach-instance-maintenance

Copying file://Users/Haengeun BI on GCP/Dashboard_5139_Test dashboard.json [Content-Type=application/json]...
Copying file://Users/Haengeun (Teach Global Liberty)/Space_16640_Haengeun (Teach Global Liberty).json [Content-Type=application/json]...
Copying file://Users/Haengeun BI on GCP/Space_16706_Haengeun BI on GCP.json [Content-Type=application/json]...
Copying file://Users/Amine Hakkou/Space_14439_Amine Hakkou.json [Content-Type=application/json]...
Copying file://Users/Karen Pang/Space_16986_Karen Pang.json [Content-Type=application/json]...
Copying file://Users/Haengeun (Teach Global Liberty)/Dashboard_5094_New Dashboard.json [Content-Type=application/json]...
Copying file://Users/Haengeun Looker/Look_12958_Best selling items.json [Content-Type=application/json]...
Copying file://Users/Haengeun Looker/Space_16681_Haengeun Looker.json [Content-Type=application/json]...
/ [8/8 files][ 46.3 KiB/ 46.3 KiB] 100% Done                                    
Operation completed over 8 object

In [ ]:
!gsutil help cp

NAME
  cp - Copy files and objects


SYNOPSIS

  gsutil cp [OPTION]... src_url dst_url
  gsutil cp [OPTION]... src_url... dst_url
  gsutil cp [OPTION]... -I dst_url



DESCRIPTION
  The ``gsutil cp`` command allows you to copy data between your local file
  system and the cloud, within the cloud, and between
  cloud storage providers. For example, to upload all text files from the
  local directory to a bucket, you can run:

    gsutil cp *.txt gs://my-bucket

  You can also download data from a bucket. The following command downloads
  all text files from the top-level of a bucket to your current directory:

    gsutil cp gs://my-bucket/*.txt .

  You can use the ``-n`` option to prevent overwriting the content of
  existing files. The following example downloads text files from a bucket
  without clobbering the data in your directory:

    gsutil cp -n gs://my-bucket/*.txt .

  Use the ``-r`` option to copy an entire directory tree.
  For example, to upload the directory tree ``dir``

# Delete users

In [ ]:
for uid in users_to_delete:
    
    print(uid)
    response = sdk.user(
            user_id= uid,
            fields="email")
    print(response.email)   
    
    #delete users
    # response = sdk.delete_user(uid)
    # print("User successfully deleted")

19371
haengeun+teach+GlobalLiberty2@google.com
19811
karenpang@looker.com
